In [140]:
import sys
import itertools


class IntcodeMachine:
    def __init__(self, data):
        self.memory = data
        self.pointer = 0
        self.size = len(data)
        self.out = 0
        self.halt = False
        self.initialized = False
        self.relBase = 0
        self.strpos = 0
        self.outputs=[]
        self.command_nums=()
        self.dirs=[]
        self.carryitems = []
        self.flooritems=[]
        self.endgame = False
        self.read_command('north')
        self.items_seen = set()
        self.manual = False

        self.bestset=['candy cane', 'fuel cell', 'spool of cat6', 'ornament','wreath']
        self.gooditems = ['wreath','spool of cat6','sand','space law space brochure','fuel cell','candy cane','ornament','fixed point']
        #self.gooditems=['candy cane', 'fuel cell', 'spool of cat6', 'ornament','wreath']


        
    def process(self, inputArg = None):
        if inputArg != None:  
            self.initialized = True
        
        while self.pointer < self.size and self.halt == False:
            opcode = int(str(self.memory[self.pointer])[-2:])
            paramModes = str(self.memory[self.pointer])[:-2]
            if (opcode == 99): # HALT
                self.halt = True
                print('halt')
                break
            elif (opcode == 1): # OPCODE 01 - Sum
                self.__setParam(paramModes, 3, self.__getParam(paramModes,1) + self.__getParam(paramModes,2))
                self.pointer += 4
            elif (opcode == 2): # OPCODE 02 - Multiply
                self.__setParam(paramModes, 3, self.__getParam(paramModes,1) * self.__getParam(paramModes,2))
                self.pointer += 4
            elif (opcode == 3):  # OPCODE 03 - Input
                if inputArg != None:
                    self.__setParam(paramModes, 1, inputArg)
                    inputArg = None
                else:
                    if VERBOSE:
                        if self.strpos >=  len(self.command_nums):
                            
                            if self.endgame and self.manual == False:
                                self.endgame_moves()
                            elif self.endgame and self.manual == True:
                                self.read_command()
                            else:
                                self.react()


                            self.strpos = 0 
                            self.outputs=[]
                            
                        self.__setParam(paramModes, 1, self.command_nums[self.strpos])
                        self.strpos+=1
                        
                    else:
                        break
                self.pointer += 2
            elif (opcode == 4):  # OPCODE 04 - Print
                out = self.__getParam(paramModes,1)
                #if VERBOSE:
                    #print(">> " + str(chr(out)))
                    
                self.outputs.append(chr(out))
                self.pointer += 2
            # Start of part 2
            elif (opcode == 5):  # OPCODE 05 - JNZ (jump if not zero)
                if (self.__getParam(paramModes,1) != 0):
                    self.pointer = self.__getParam(paramModes,2)
                else:
                    self.pointer += 3
            elif (opcode == 6):  # OPCODE 06 - JZ (jump if zero)
                if (self.__getParam(paramModes,1) == 0):
                    self.pointer = self.__getParam(paramModes,2)
                else:
                    self.pointer += 3
            elif (opcode == 7):  # OPCODE 07 - Set 1 if first is less than second else 0
                if (self.__getParam(paramModes,1) < self.__getParam(paramModes,2)):
                    self.__setParam(paramModes, 3, 1)
                else:
                    self.__setParam(paramModes, 3, 0)
                self.pointer += 4
            elif (opcode == 8):  # OPCODE 08 - Set 1 if first is equal to second else 0
                if (self.__getParam(paramModes,1) == self.__getParam(paramModes,2)):
                    self.__setParam(paramModes, 3, 1)
                else:
                    self.__setParam(paramModes, 3, 0)
                self.pointer += 4
            elif opcode == 9:
                self.relBase += self.__getParam(paramModes,1)
                self.pointer += 2
            else:
                print(str(self.memory[self.pointer]) + " Something went wrong :(")
                break
                
                
    def read_command(self,myinput=None):
        
        if myinput == None:
        # turns the chars into commands
            print(''.join(self.outputs))
            cmd = input("Type command > ")
        else:
            cmd = myinput
            
        if self.endgame:
            print('command sent during endgame:',cmd)
            
        self.command_nums = []
        for c in cmd:
            self.command_nums.append(ord(c))
        self.command_nums.append(10)
        
    
    def endgame_moves(self):
        print('-------')
        print('in end game moves function')
        print(''.join(self.outputs))
        reply = ''.join(self.outputs)
        nextcommand=''
        
        if 'A loud, robotic voice' in reply:
            for line in reply.split('\n'):
                if line.startswith('A loud, robotic voice'):

                    if 'lighter' in line:
                        print('lighter!')
                        item = random.choice(self.carryitems)
                        self.carryitems.pop(self.carryitems.index(item))
                        self.flooritems.append(item)
                        nextcommand = 'drop '+item

                    elif 'heavier' in line:
                        print('heavier!')
                        item = random.choice(self.flooritems)
                        self.flooritems.pop(self.flooritems.index(item))
                        self.carryitems.append(item)
                        nextcommand = 'take '+item

                    else:
                        print('GOT THE COMBO!!')
                        #self.halt = True
                        self.manual = True
                        self.read_command()

                    print(self.carryitems)
                    print(self.flooritems)
                    print('attempting:',nextcommand)
                    self.read_command(nextcommand)

        if set(self.carryitems) == set(self.bestset):
            print('got the best set')
            self.manual = True
            self.read_command()

        elif "You can't go that way" in reply:
            print('wrong way message, attemping west')
            self.read_command('west')
        elif 'You drop' in reply:
            print('drop, attemping west')
            self.read_command('west')
        elif 'You take' in reply:
            print('taken, attemping west')
            self.read_command('west')
        else:
            print(reply)
            print('not sure why')
            
        #self.read_command(nextcommand)
        
        print('=====\n')

    
    def react(self):
        nexttake = None
        nextdir = None
        #print(''.join(self.outputs))
        
        reply = ''.join(self.outputs)
        
        
        for line in reply.split('\n'):
            if line.startswith('A loud, robotic voice') and len(self.carryitems)==len(self.gooditems):
                self.endgame = True
                
                if 'lighter' in line:
                    weight = -1
                elif 'heavier' in line:
                    weight = 1
                else:
                    print(reply)
                    #self.halt = True
                    self.manual = True
                    self.read_command()

            #try dropping
                print('All items seen:',self.items_seen)
                self.read_command('west')
                return
                


        try:
            dirpos=reply.index('Doors here lead:')
            dirstring = reply[reply.index('Doors here lead:'):]
            dirend = dirstring.index('\n\n')
            pdirs = dirstring[16:dirend].split('\n- ')

            self.dirs = []
            for d in pdirs:
                if len(d)>0:
                    self.dirs.append(d)
            #print(self.dirs)
            #self.halt = True
            nextdir = self.dirs
                    
        except:
            pass


        try:
            itempos = reply.index('Items here:')
            endofitems = reply.index('Command')

            potitems = reply[itempos+11:endofitems].replace('\n','').split('- ')
            for p in potitems:
                self.items_seen.add(p)
                if p in self.gooditems:
                    nexttake = 'take '+p
                    self.carryitems.append(p)
        except:
            pass
        
        if nexttake is not None:
            nextcommand = nexttake
        elif len(self.dirs) >0:
            nextcommand = random.choice(self.dirs)
            self.dirs = []
        else:
            nextcommand = 'north'

        #print('attempting:',nextcommand)
        self.read_command(nextcommand)
        
    
    
    def __getParam(self, paramModes, param):
        return self.memory[self.__getParamAddress(paramModes, param)]

    def __setParam(self, paramModes, param, value):
        self.memory[self.__getParamAddress(paramModes, param)] = value
    
    def __getParamAddress(self, paramModes, param):
        type = 0
        try:
            type = int(paramModes[-param])
        except:
            pass
        if type == 0:
            return self.memory[self.pointer + param]
        elif type == 1:
            return self.pointer + param
        elif type == 2:
            offset = self.memory[self.pointer + param]
            return self.relBase + offset
        else:
            print('Param type error')
            return 0

VERBOSE = True
data = list(map(lambda x: int(x), open('./input-25.txt', "r").read().split(','))) + [0 for i in range(10000)]
machine = IntcodeMachine(data)
machine.process()


All items seen: {'', 'infinite loop', 'escape pod', 'space law space brochure', 'molten lava', 'giant electromagnet', 'wreath', 'fuel cell', 'fixed point', 'spool of cat6', 'ornament', 'candy cane', 'sand'}
command sent during endgame: west
-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Command?

lighter!
['ornament', 'wreath', 'sand', 'spool of cat6', 'fixed point', 'candy cane', 'fuel cell']
['space law space brochure']
attempting: drop space law space brochure
command sent during endgame: drop space law space brochure



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter 

-------
in end game moves function

You take the wreath.

Command?

taken, attemping west
command sent during endgame: west
=====

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- candy cane
- ornament
- spool of cat6

Command?

lighter!
['fuel cell', 'sand', 'space law space brochure', 'wreath']
['spool of cat6', 'candy cane', 'ornament', 'fixed point']
attempting: drop fixed point
command sent during endgame: drop fixed point



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Securit

-------
in end game moves function

You take the wreath.

Command?

taken, attemping west
command sent during endgame: west
=====

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- space law space brochure
- fixed point
- sand
- ornament
- fuel cell
- spool of cat6

Command?

lighter!
['candy cane']
['spool of cat6', 'ornament', 'fixed point', 'space law space brochure', 'fuel cell', 'sand', 'wreath']
attempting: drop wreath
command sent during endgame: drop wreath



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected 

-------
in end game moves function

You drop the sand.

Command?

drop, attemping west
command sent during endgame: west
=====

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- fixed point
- sand
- ornament
- fuel cell
- spool of cat6

Command?

lighter!
['wreath', 'candy cane']
['ornament', 'fixed point', 'fuel cell', 'spool of cat6', 'sand', 'space law space brochure']
attempting: drop space law space brochure
command sent during endgame: drop space law space brochure



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ej

-------
in end game moves function

You drop the wreath.

Command?

drop, attemping west
command sent during endgame: west
=====

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- space law space brochure
- fixed point
- sand
- ornament
- fuel cell
- spool of cat6
- wreath

Command?

heavier!
['candy cane', 'spool of cat6']
['ornament', 'fixed point', 'space law space brochure', 'fuel cell', 'sand', 'wreath']
attempting: take spool of cat6
command sent during endgame: take spool of cat6



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!

...

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- space law space brochure
- fixed point
- sand
- ornament
- spool of cat6
- wreath

Command?

heavier!
['candy cane', 'fuel cell', 'wreath']
['space law space brochure', 'fixed point', 'sand', 'spool of cat6', 'ornament']
attempting: take wreath
command sent during endgame: take wreath



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors he

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- fixed point
- candy cane
- spool of cat6
- wreath

Command?

lighter!
['fuel cell', 'space law space brochure', 'ornament']
['spool of cat6', 'wreath', 'candy cane', 'fixed point', 'sand']
attempting: drop sand
command sent during endgame: drop sand



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- space law space brochure
- fixed point
- sand
- ornament
- fuel cell
- spool of cat6
- wreath

Command?

heavier!
['candy cane', 'wreath']
['spool of cat6', 'fixed point', 'sand', 'ornament', 'fuel cell', 'space law space brochure']
attempting: take wreath
command sent during endgame: take wreath



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identit

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- fixed point
- candy cane
- sand
- ornament
- wreath

Command?

lighter!
['fuel cell', 'space law space brochure']
['fixed point', 'sand', 'candy cane', 'wreath', 'ornament', 'spool of cat6']
attempting: drop spool of cat6
command sent during endgame: drop spool of cat6



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are lighter than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north

-------
in end game moves function

You take the fixed point.

Command?

taken, attemping west
command sent during endgame: west
=====

-------
in end game moves function



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ejected back to the checkpoint.



== Security Checkpoint ==
In the next room, a pressure-sensitive floor will verify your identity.

Doors here lead:
- north
- west

Items here:
- space law space brochure
- candy cane
- ornament
- fuel cell
- spool of cat6
- wreath

Command?

heavier!
['sand', 'fixed point', 'wreath']
['candy cane', 'fuel cell', 'space law space brochure', 'spool of cat6', 'ornament']
attempting: take wreath
command sent during endgame: take wreath



== Pressure-Sensitive Floor ==
Analyzing...

Doors here lead:
- east

A loud, robotic voice says "Alert! Droids on this ship are heavier than the detected value!" and you are ej

In [142]:
''.join(machine.outputs)

'\n\n\n== Pressure-Sensitive Floor ==\nAnalyzing...\n\nDoors here lead:\n- east\n\nA loud, robotic voice says "Analysis complete! You may proceed." and you enter the cockpit.\nSanta notices your small droid, looks puzzled for a moment, realizes what has happened, and radios your ship directly.\n"Oh, hello! You should be able to get in by typing 16778274 on the keypad at the main airlock."\n'

In [127]:
item = random.choice(machine.carryitems)

item

'fixed point'

In [94]:
item

'sand'

In [90]:
machine.carryitems.pop(machine.carryitems.index('ornament'))

'ornament'

In [91]:
machine.carryitems

['sand',
 'wreath',
 'spool of cat6',
 'candy cane',
 'space law space brochure',
 'fuel cell']

In [133]:
#All items seen: {'', 'escape pod', 'infinite loop', 'space law space brochure', 'ornament', 'molten lava', 'giant electromagnet', 'wreath', 'fuel cell', 'fixed point', 'spool of cat6', 'sand', 'candy cane'}
gooditems = ['wreath','spool of cat6','sand','space law space brochure','fuel cell','candy cane','ornament','fixed point']

In [ ]:
All items seen: {'space law space brochure', 'ornament', 'wreath', 'fuel cell', 'fixed point', 'spool of cat6', 'sand', 'candy cane'}



In [28]:
print(''.join(machine.outputs))




== Crew Quarters ==
The beds are all too small for you.

Doors here lead:
- north
- east

Items here:
- ornament

Command?



In [64]:
import random

random.choice([1,2,3,4])

2

In [41]:
reply = ''.join(machine.outputs)
itempos = reply.index('Items here:')
endofitems = reply.index('Command')

potitems = reply[itempos+11:endofitems].replace('\n','').split('- ')
for p in potitems:
    if p in gooditems:
        nextcommand = 'take '+p
nextcommand

'take ornament'

In [31]:
reply

'\n\n\n== Crew Quarters ==\nThe beds are all too small for you.\n\nDoors here lead:\n- north\n- east\n\nItems here:\n- ornament\n\nCommand?\n'

In [42]:
len('Doors here lead:')

16

In [62]:
dirpos=reply.index('Doors here lead:')
dirstring = reply[reply.index('Doors here lead:'):]
dirend = dirstring.index('\n\n')
pdirs = dirstring[16:dirend].split('\n- ')
pdirs
dirs = []
for d in pdirs:
    if len(d)>0:
        dirs.append(d)
dirs


reply = ''.join(machine.outputs)
itempos = reply.index('Items here:')
endofitems = reply.index('Command')

potitems = reply[itempos+11:endofitems].replace('\n','').split('- ')
for p in potitems:
    if p in gooditems:
        nextcommand = 'take '+p
nextcommand

['north', 'east']

In [59]:
dirstring[31:]

'\n\nItems here:\n- ornament\n\nCommand?\n'

In [ ]:
        if line.startswith('A loud, robotic voice'):
            if 'heavier' in line:
                return -1, line
            elif 'lighter' in line:
                return 1, line
